In [80]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.rdd.RDD
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .master("local[*]").appName("Validator")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Creating SparkSession


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import $ivy.$                                

import $ivy.$                              


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.rdd.RDD

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@3832e54c
import spark.implicits._

appID: String = "local-1557362230978"

### Collecting datasets...
Polygons from several states were collected for both source and target in WKT format. They are available at: https://github.com/aocalderon/RIDIR/tree/master/Datasets/AreaInterpolateValidation.

In [81]:
import sys.process._

val path = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
s"ls -lah ${path}" #| "grep wkt" !

-rw-rw-r-- 1 acald013 acald013  14M May  8 18:47 CA_source.wkt
-rw-rw-r-- 1 acald013 acald013  18M May  8 18:47 CA_target.wkt
-rw-rw-r-- 1 acald013 acald013 1.3M May  8 12:00 MD_source.wkt
-rw-rw-r-- 1 acald013 acald013 1.6M May  8 12:00 MD_target.wkt
-rw-rw-r-- 1 acald013 acald013 2.3M May  8 18:34 NY_source.wkt
-rw-rw-r-- 1 acald013 acald013 4.5M May  8 18:34 NY_target.wkt
-rw-rw-r-- 1 acald013 acald013 4.6M May  8 19:24 PA_source.wkt
-rw-rw-r-- 1 acald013 acald013 7.4M May  8 19:24 PA_target.wkt
-rw-rw-r-- 1 acald013 acald013  12M May  8 19:33 TX_source.wkt
-rw-rw-r-- 1 acald013 acald013  20M May  8 19:33 TX_target.wkt


import sys.process._


path: String = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
res80_2: Int = 0

For each set of source & target, we run the corresponding script:

* For GeoPandas implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Validation/GeoPandas_interpolate_table_tester.py
* For GeoSpark implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Areal/src/main/scala/GeoSpark_area_interpolate_tester.scala

Each script save the results to disk for further analysis (files are also available in the same repo).

In [82]:
val path = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
s"ls -lah ${path}" #| "grep tsv" !

-rw-rw-r-- 1 acald013 acald013 105K May  8 19:01 CA_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 105K May  8 18:49 CA_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013  13K May  8 17:13 MD_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013  13K May  8 17:05 MD_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013  30K May  8 18:37 NY_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013  30K May  8 18:43 NY_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013  37K May  8 19:30 PA_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013  37K May  8 19:26 PA_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013  69K May  8 19:44 TX_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013  69K May  8 19:35 TX_geospark_test.tsv


path: String = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
res81_1: Int = 0

### Set a particular state to run the validation...

In [83]:
val state = "TX"

state: String = "TX"

### Reading results from geopandas implementation...

In [84]:
val geopandas = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geopandas_test.tsv")
    .repartition(1).toDF("pandas").orderBy("pandas")
geopandas.show()
geopandas.count()

+--------------------+
|              pandas|
+--------------------+
|0.000476079421227...|
|0.000563932359796...|
|0.000896799723312...|
|0.001385847179852...|
|0.001410064041914...|
|0.001533656258423...|
|0.002412955666533...|
|0.002422142098462...|
|0.002456665434444205|
|0.002610379311994...|
|0.005572083978785185|
|0.006011155024625339|
|0.010913706279911804|
| 0.01324190605209407|
| 0.01346655366362276|
|0.014593566667149829|
|0.015896933429055075|
|0.020840455733341787|
|0.021429709116567837|
|0.022633093861060563|
+--------------------+
only showing top 20 rows



geopandas: Dataset[Row] = [pandas: string]
res83_2: Long = 3816L

### Reading results from geospark implementation...

In [85]:
val geospark = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geospark_test.tsv")
    .repartition(1).toDF("spark").orderBy("spark")
geospark.show()
geospark.count()

+--------------------+
|               spark|
+--------------------+
|0.001385847179852...|
|0.001410064041914...|
|0.001533656258423...|
|0.002412955666533939|
|0.002422139852092...|
|0.002456665434444...|
|0.002610379311994...|
|0.005566232568193...|
|0.006011155024625326|
|0.010913706279911806|
|0.013241906052094026|
|0.013474764977294182|
| 0.01459356666714996|
|0.015902872572923107|
| 0.02084045573334167|
|0.021429709116568343|
|0.022633093861060955|
|0.024338736848408732|
|0.024918913495777243|
|  0.0278524967657295|
+--------------------+
only showing top 20 rows



geospark: Dataset[Row] = [spark: string]
res84_2: Long = 3816L

### Merging both result sets...

In [86]:
val popPandas = geopandas.rdd.map(_.getString(0).toDouble).sortBy(f => f)
val popSpark  = geospark.rdd.map(_.getString(0).toDouble).sortBy(f => f)

import org.apache.spark.sql.functions._
val pops = popPandas.zip(popSpark)

pops.toDF("pandas", "spark")
    .withColumn("diff", abs($"pandas" - $"spark"))
    .orderBy(desc("diff"))
    .show(50, truncate = false)

+------------------+------------------+------------------+
|pandas            |spark             |diff              |
+------------------+------------------+------------------+
|14868.94954038966 |14753.26688174401 |115.68265864565001|
|16660.553521568356|16551.84360581831 |108.70991575004518|
|25898.22571552311 |25797.12682158911 |101.09889393399862|
|20389.57960944831 |20301.894996666073|87.68461278223549 |
|16076.09052618457 |16159.399258853131|83.30873266856179 |
|11636.785158459279|11716.888642641963|80.10348418268404 |
|15597.522378113714|15523.920374786803|73.60200332691056 |
|10860.188767826672|10911.093048933839|50.90428110716675 |
|15645.484865502869|15597.522378113734|47.962487389135276|
|11492.45899176329 |11540.365293284292|47.906301521003115|
|11229.512726131841|11276.965251504811|47.45252537296983 |
|10754.81366116398 |10796.289439325381|41.47577816140074 |
|14753.26688174398 |14712.842769753795|40.42411199018534 |
|11737.64975006734 |11777.541684696786|39.89193462944604

popPandas: RDD[Double] = MapPartitionsRDD[1252] at sortBy at cmd85.sc:1
popSpark: RDD[Double] = MapPartitionsRDD[1271] at sortBy at cmd85.sc:2
import org.apache.spark.sql.functions._

pops: RDD[(Double, Double)] = ZippedPartitionsRDD2[1272] at zip at cmd85.sc:5

### Running some metrics to test similarity...

In [87]:
val reg = new RegressionMetrics(pops)
println(s"R\u00B2:   ${reg.r2}")
println(s"MAE:  ${reg.meanAbsoluteError}")
println(s"MSE:  ${reg.meanSquaredError}")
println(s"RMSE: ${reg.rootMeanSquaredError}")

R²:   0.9999970388898133
MAE:  3.3044751937679075
MSE:  48.19765286541765
RMSE: 6.942452942974669


reg: RegressionMetrics = org.apache.spark.mllib.evaluation.RegressionMetrics@1a438ae4